# Run Indexamajig with options for peakfinding, indexing and integration

In [ ]:
from gandalf_radial_iterator import gandalf_iterator

geomfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTAsim.geom"                  # .geom file
cellfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTA.cell"                     # .cell file

# geomfile_path = "/home/bubl3932/files/LTA_sim/LTAsim.geom"                  # .geom file
# cellfile_path = "/home/bubl3932/files/LTA_sim/LTA.cell"                     # .cell file

input_path =   "/Users/xiaodong/Desktop/simulations/LTA/simulation-24"      # .h5 folder will also be output folder
# input_path =   "/home/bubl3932/files/LTA_sim/simulation-43"               # .h5 folder will also be output folder

output_file_base = "LTA"    # output files will be named output_file_base_xcoord_ycoord.h5

num_threads = 8             # number of CPU threads to use
x, y = 512.5, 512.5         # initial beam center from where iterations will start

"""Define the grid and radius for iterations.
As example max_radius = 1, step = 0.2 will give 81 iterations.
Iterations will start at the center and move radially outwards.
"""
max_radius = 1              # maximum radius in pixels
step = 0.2                  # grid granularity in pixels

extra_flags=[
# PEAKFINDING
"--no-revalidate",
"--no-half-pixel-shift",
"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-biggest-pix=1",
"--min-sig=3",
"--local-bg-radius=10",
# INDEXING
"--indexing=xgandalf",
"--tolerance=10,10,10,5",
"--no-refine",
# "--no-retry",
"--xgandalf-sampling-pitch=5",
"--xgandalf-grad-desc-iterations=1",
"--xgandalf-tolerance=0.02",
# "--xgandalf-no-deviation-from-provided-cell",
# INTEGRATION
"--integration=rings",
"--int-radius=4,5,9",
"--fix-profile-radius=70000000",
# OUTPUT
"--no-non-hits-in-stream",
]

"""Examples of extra flags(see crystfel documentation https://www.desy.de/~twhite/crystfel/manual-indexamajig.html):"""

""" Basic options
"--highres=n",
"--no-image-data",
"""

""" Peakfinding
"--peaks=cxi",
"--peak-radius=inner,middle,outer",
"--min-peaks=n",
"--median-filter=n",
"--filter-noise",
"--no-revalidate",
"--no-half-pixel-shift",

"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-peak-pix=6",
"--min-snr-biggest-pix=1",
"--min-sig=9",
"--min-peak-over-neighbour=5",
"--local-bg-radius=5",

"--peaks=peakfinder8",
"--threshold=45",
"--min-snr=3",
"--min-pix-count=3",
"--max-pix-count=500",
"--local-bg-radius=9",
"--min-res=30",
"--max-res=500",
"""

""" Indexing
"--indexing=xgandalf",

"--tolerance=tol"
"--no-check-cell",
"--no-check-peaks",
"--multi",
"--no-retry",
"--no-refine",

"--xgandalf-sampling-pitch=n"
"--xgandalf-grad-desc-iterations=n"
"--xgandalf-tolerance=n"
"--xgandalf-no-deviation-from-provided-cell"
"--xgandalf-max-lattice-vector-length=n"
"--xgandalf-min-lattice-vector-length=n"
"--xgandalf-max-peaks=n"
"--xgandalf-fast-execution"
"""

""" Integration
"--fix-profile-radius=n",
"--fix-divergence=n",
"--integration=rings",
"--int-radius=4,5,10",
"--push-res=n",
"--overpredict",
"--cell-parameters-only",
"""

""" Output
"--no-non-hits-in-stream",
"--no-peaks-in-stream",
"--no-refls-in-stream",
"--serial-offset
"""

gandalf_iterator(x, y, geomfile_path, cellfile_path, input_path, output_file_base, num_threads, max_radius=max_radius, step=step, extra_flags=extra_flags)


'list.lst' has been created with 1 entries at /Users/xiaodong/Desktop/simulations/LTA/simulation-24/list.lst


Processing XY pairs:   0%|          | 0/77 [00:00<?, ?it/s]

Running for x=-512.5, y=-512.5


This is what I understood your unit cell to be:
cubic P, right handed.
a      b      c            alpha   beta  gamma
 11.87  11.87  11.87 A     90.00  90.00  90.00 deg
List of indexing methods:
   0: xgandalf-nolatt-cell      (xgandalf using cell parameters as prior information)
Indexing parameters:
                  Check unit cell parameters: on
                        Check peak alignment: on
                   Refine indexing solutions: off
 Multi-lattice indexing ("delete and retry"): off
                              Retry indexing: on
28 images processed, 28 hits (100.0%), 27 indexable (96.4% of hits, 96.4% overall), 27 crystals, 5.5 images/sec.
52 images processed, 52 hits (100.0%), 50 indexable (96.2% of hits, 96.2% overall), 50 crystals, 4.7 images/sec.
83 images processed, 83 hits (100.0%), 79 indexable (95.2% of hits, 95.2% overall), 79 crystals, 5.9 images/sec.
Waiting for the last patterns to be processed...
Final: 100 images processed, 100 hits (100.0%), 94 indexable (9

Running for x=-512.7, y=-512.5


...

Running for x=-512.9, y=-512.9


19 images processed, 19 hits (100.0%), 19 indexable (100.0% of hits, 100.0% overall), 19 crystals, 3.8 images/sec.
37 images processed, 37 hits (100.0%), 34 indexable (91.9% of hits, 91.9% overall), 34 crystals, 3.6 images/sec.
53 images processed, 53 hits (100.0%), 47 indexable (88.7% of hits, 88.7% overall), 47 crystals, 3.1 images/sec.
71 images processed, 71 hits (100.0%), 62 indexable (87.3% of hits, 87.3% overall), 62 crystals, 3.3 images/sec.
Waiting for the last patterns to be processed...
94 images processed, 94 hits (100.0%), 83 indexable (88.3% of hits, 88.3% overall), 83 crystals, 4.6 images/sec.
Final: 100 images processed, 100 hits (100.0%), 86 indexable (86.0% of hits, 86.0% overall), 86 crystals.
Processing XY pairs:  29%|██▊       | 22/77 [08:53<26:38, 29.06s/it]This is what I understood your unit cell to be:
cubic P, right handed.
a      b      c            alpha   beta  gamma
 11.87  11.87  11.87 A     90.00  90.00  90.00 deg
List of indexing methods:
   0: xgandalf-

Running for x=-512.9, y=-512.1


18 images processed, 18 hits (100.0%), 16 indexable (88.9% of hits, 88.9% overall), 16 crystals, 3.4 images/sec.
34 images processed, 34 hits (100.0%), 29 indexable (85.3% of hits, 85.3% overall), 29 crystals, 3.1 images/sec.
51 images processed, 51 hits (100.0%), 41 indexable (80.4% of hits, 80.4% overall), 41 crystals, 3.4 images/sec.
67 images processed, 67 hits (100.0%), 52 indexable (77.6% of hits, 77.6% overall), 52 crystals, 2.9 images/sec.
82 images processed, 82 hits (100.0%), 59 indexable (72.0% of hits, 72.0% overall), 59 crystals, 3.0 images/sec.
Waiting for the last patterns to be processed...
98 images processed, 98 hits (100.0%), 69 indexable (70.4% of hits, 70.4% overall), 69 crystals, 3.1 images/sec.
Final: 100 images processed, 100 hits (100.0%), 70 indexable (70.0% of hits, 70.0% overall), 70 crystals.
Processing XY pairs:  30%|██▉       | 23/77 [09:25<26:53, 29.89s/it]This is what I understood your unit cell to be:
cubic P, right handed.
a      b      c            a

Running for x=-512.1, y=-512.9


# Evaluate the IQM with chosen weights for all frames and all index results

In [2]:
from automate_evaluation import automate_evaluation

# Enter folder with stream file results from indexamajig. 
# Note that ALL stream files in the folder will be processed.
stream_file_folder = "/Users/xiaodong/Desktop/simulations/LTA/simulation-33" 

weights_list = [
    (1, 1, 1, 1, 1, 1)
]

"""
Each weight corresponds to one of the six metrics used in calculating the combined IQM value.
The combined IQM is computed by first normalizing each metric across all stream files, then 
multiplying each normalized metric by its assigned weight, and finally summing the results.
The order (or keys) of the weights must match the following metrics:

- 'weighted_rmsd'
- 'fraction_outliers'
- 'length_deviation'
- 'angle_deviation'
- 'peak_ratio'
- 'percentage_indexed'

Multiple weight combinations can be specified if needed.
"""

automate_evaluation(stream_file_folder, weights_list, indexing_tolerance=2)

Evaluating multiple stream files with weights: (1, 1, 1, 1, 1, 1)


Processing chunks in LTA_-512.3_-512.4.stream: 100%|██████████| 432/432 [00:38<00:00, 11.22chunk/s] 9.75chunk/s]
Processing chunks in LTAsim_xgandalf_-512.5_-512.5.stream:  13%|█▎        | 536/4186 [00:51<06:58,  8.73chunk/s]Process SpawnProcess-2:
Processing chunks in LTAsim_xgandalf_-512.5_-512.5.stream:  13%|█▎        | 536/4186 [00:51<05:47, 10.51chunk/s]
Traceback (most recent call last):
  File "/opt/anaconda3/envs/pyxem-env/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/pyxem-env/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/pyxem-env/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/opt/anaconda3/envs/pyxem-env/lib/python3.10/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/opt/anaconda3/envs/pyxem-env/lib/python3.10/

KeyboardInterrupt: 

# Merge the best results stream file

In [ ]:
from merge import merge

stream_file = ".stream"
pointgroup = "m-3m"
num_threads = 24
iterations = 5

output_dir = merge(
    stream_file,
    pointgroup=pointgroup,
    num_threads=num_threads,
    iterations=iterations,
)

if output_dir is not None:
    print("Merging done. Results are in:", output_dir)

# Convert to shelx compatible .hkl

In [ ]:

import os 
from convert_hkl_crystfel_to_shelx import convert_hkl_crystfel_to_shelx 
convert_hkl_crystfel_to_shelx(os.path.join(output_dir,"crystfel.hkl"))

# Convert to mtz

In [ ]:
from convert_hkl_to_mtz import convert_hkl_to_mtz
cellfile_path = ""
convert_hkl_to_mtz(output_dir, cellfile_path=cellfile_path)